In [68]:
import copy
from typing import Dict, Tuple

import numpy as np
import pandas as pd
import sklearn
import sklearn.model_selection
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, Subset, DataLoader
from tqdm.notebook import tqdm

from recommend.utils import PROJ_ROOT

In [8]:
ratings = pd.read_pickle(f"{PROJ_ROOT}/data/ratings.pkl")

In [9]:
ratings.shape

(2797751, 5)

In [10]:
def filter_ratings(ratings: pd.DataFrame, min_user_ratings: int, min_movie_ratings: int) -> pd.DataFrame:
    ratings = ratings.dropna()

    user_rating_count = ratings.groupby("username").size()
    worthy_users = set(user_rating_count[user_rating_count >= min_user_ratings].index)
    ratings = ratings.loc[ratings.username.isin(worthy_users)]

    movie_rating_count = ratings.groupby("movie_id").size()
    worthy_movies = set(movie_rating_count[movie_rating_count >= min_movie_ratings].index)
    ratings = ratings.loc[ratings.movie_id.isin(worthy_movies)]
    
    return ratings

In [11]:
ratings = filter_ratings(ratings, 10, 10)

In [12]:
ratings.shape

(2722362, 5)

In [13]:
ratings_train, ratings_test = sklearn.model_selection.train_test_split(
    ratings,
    test_size=0.1,
    shuffle=True,
    random_state=0,
)

ratings_train, ratings_valid = sklearn.model_selection.train_test_split(
    ratings_train,
    test_size=0.1,
    shuffle=True,
    random_state=0,
)

In [14]:
idx2movie = pd.Series(ratings.movie_id.unique())
movie2idx = pd.Series(idx2movie.index.values, index=idx2movie.values)
movie2idx.head()

230421-houbicky                       0
10789-prvni-liga                      1
235032-yes-man                        2
234754-chlapec-v-pruhovanem-pyzamu    3
301717-nejvetsi-showman               4
dtype: int64

In [15]:
idx2user = pd.Series(ratings.username.unique())
user2idx = pd.Series(idx2user.index.values, index=idx2user.values)
user2idx.head()


kinghome     0
SimonShot    1
blackend     2
LCH          3
knoxville    4
dtype: int64

In [80]:
class Model(torch.nn.Module):

    def __init__(self, num_features: int, num_movies: int, num_users: int, seed: int = 42):
        super().__init__()
        random_gen = np.random.default_rng(seed)

        self.movies_weights = torch.nn.Embedding.from_pretrained(torch.from_numpy(
            random_gen.standard_normal((num_movies, num_features), dtype=np.float32).clip(-2, 2)
        ))

        self.users_weights = torch.nn.Embedding.from_pretrained(torch.from_numpy(
            random_gen.standard_normal((num_users, num_features), dtype=np.float32).clip(-2, 2)
        ))

        self.movies_biases = torch.nn.Parameter(torch.from_numpy(
            random_gen.standard_normal((num_movies,), dtype=np.float32).clip(-2, 2)
        ))

        self.users_biases = torch.nn.Parameter(torch.from_numpy(
            random_gen.standard_normal((num_users,), dtype=np.float32).clip(-2, 2)
        ))
        

    def forward(self, movie_idx, user_idx) -> torch.Tensor:
        m_w = self.movies_weights(movie_idx)
        m_b = self.movies_biases[movie_idx]
        u_w = self.users_weights(user_idx)
        u_b = self.users_biases[user_idx]
        return torch.sigmoid((m_w * u_w).sum(-1) + m_b + u_b)

In [99]:
model = Model(
    num_features=100,
    num_movies=len(movie2idx),
    num_users=len(user2idx)
)

In [100]:
device = 'cuda'
model = model.to(device)

In [101]:
ratings_train.head()

,username,movie_id,stars,date,comment
328863,pepo,2596-osm-a-pul,1.0,2018-04-24,"Súhlasím, že 8 1/2 je umelecký film. Je totiž..."
1601217,Melly.pro,224137-hvezdny-prach,5.0,2008-01-18,"Tak nejsem si úplně jistej, co o tom říct, pr..."
1369474,GOREGASM,223564-kazatel,2.0,2016-01-31,Béčkové vody zakalené do digitálního apokalyp...
499592,Ernie_13,308635-zpatky-ve-hre,3.0,2014-03-07,"No je to tých 70%....hrozná réžia, hrozný sce..."
2322162,Khumbac,71194-walker-texas-ranger,4.0,2012-07-14,"Z nostalgie, z recese, z úcty ke komedii dává..."


In [102]:
class RatingDataset(Dataset): 

    def __init__(self, df_ratings: pd.DataFrame, movie2idx: pd.Series, user2idx: pd.Series):
        self.df_ratings = df_ratings
        self.movie2idx = movie2idx
        self.user2idx = user2idx

    def __len__(self) -> int:
        return len(self.df_ratings)

    def __getitem__(self, idx: int) -> Tuple[int, int, float]:
        row = self.df_ratings.iloc[idx]
        idx_movie = self.movie2idx[row.movie_id]
        idx_user = self.user2idx[row.username]
        rating = (row.stars / 5.0).astype(np.float32)
        return idx_movie, idx_user, rating

In [103]:
ds_train = RatingDataset(ratings_train, movie2idx, user2idx)
ds_valid = RatingDataset(ratings_valid, movie2idx, user2idx)
ds_test = RatingDataset(ratings_test, movie2idx, user2idx)

ds_train_mini = Subset(
    ds_train,
    np.random.default_rng(0).choice(len(ds_train), len(ds_valid), replace=False)
)

In [104]:
len(ds_train), len(ds_valid), len(ds_test), len(ds_train_mini)

(2205112, 245013, 272237, 245013)

In [105]:
ds_train[0]

(1023, 2359, 0.2)

In [106]:
batch_size = 64

def collate_move(device):
    def collate(*params):
        movie_idx, user_idx, rating = torch.utils.data.default_collate(*params)
        return movie_idx.to(device), user_idx.to(device), rating.to(device)
    return collate


loader_train = DataLoader(ds_train, batch_size, collate_fn=collate_move(device))
loader_valid = DataLoader(ds_valid, batch_size, collate_fn=collate_move(device))
loader_test = DataLoader(ds_test, batch_size, collate_fn=collate_move(device))

loader_train_mini = DataLoader(ds_train_mini, batch_size, collate_fn=collate_move(device))

In [107]:
len(loader_train), len(loader_valid), len(loader_test), len(loader_train_mini)

(34455, 3829, 4254, 3829)

In [108]:
optim = torch.optim.Adam(
    params=model.parameters(),
    lr=1e-1,
)

In [114]:
class Trainer:

    def __init__(
        self,
        model: Model,
        optimizer: torch.optim.Optimizer,
    ):
        self.model = model
        self.metrics_train = []
        self.metrics_valid = []
        self.past_models = []
        self.optimizer = optimizer


    def evaluate(
        self,
        loader: DataLoader,
        tqdm_desc: str,
        tqdm_leave: bool,
    ) -> Dict[str, float]:

        metrics = {
            "mse": 0.0,
            "mae_stars": 0.0
        }

        with torch.no_grad():
            for movie_idx, user_idx, rating in tqdm(loader, desc=tqdm_desc, leave=tqdm_leave, position=1):
                pred = model(movie_idx, user_idx)
                metrics["mse"] += F.mse_loss(pred, rating).cpu().item()
                metrics["mae_stars"] += F.l1_loss(pred * 5, rating * 5).cpu().item()
        return {name: value / len(loader) for name, value in metrics.items()}

    
    def train(
        self,
        loader_train: DataLoader,
        loader_train_mini: DataLoader,
        loader_valid: DataLoader,
        patience: int = 5,
        max_epochs: int = 25,
        evaluate_every_n_steps: int = 12500,
        early_stopping_metric: str = "mae_stars",
    ):
        step = 0
        for epoch in range(max_epochs):
            for movie_idx, user_idx, rating in tqdm(loader_train, desc=f"epoch {epoch}", position=0):
                pred = model(movie_idx, user_idx)
                F.mse_loss(pred, rating).backward()
                self.optimizer.step()
                self.optimizer.zero_grad()
                if step % evaluate_every_n_steps == 0:
                    self.metrics_train.append(self.evaluate(loader_train_mini, "train evaluation", tqdm_leave=False))
                    self.metrics_valid.append(self.evaluate(loader_valid, "validation evaluation", tqdm_leave=False))
                    print("train metrics:     ", self.metrics_train[-1])
                    print("validation metrics:", self.metrics_valid[-1])
                    self.past_models.append(copy.deepcopy(model).cpu())
                    if len(self.metrics_train) >= patience:
                        self.past_models.pop(0)
                        last_n = pd.DataFrame(self.metrics_train[-patience:])[early_stopping_metric]
                        if last_n.argmin() == 0:
                            device = list(model.parameters())[0].device
                            self.model = self.past_models[0].to(device)
                            return self.model
                step += 1
        return self.model


In [110]:
trainer = Trainer(model, optim)

In [111]:
trainer.train(loader_train, loader_train_mini, loader_valid)

epoch 0:   0%|          | 0/34455 [00:00<?, ?it/s]

train evaluation:   0%|          | 0/3829 [00:00<?, ?it/s]

validation evaluation:   0%|          | 0/3829 [00:00<?, ?it/s]

train metrics:      {'mse': 0.31293469379841116, 'mae_stars': 2.321519985339697}
validation metrics: {'mse': 0.31249594485270943, 'mae_stars': 2.3195831577035024}


train evaluation:   0%|          | 0/3829 [00:00<?, ?it/s]

...

epoch 13:   0%|          | 0/34455 [00:00<?, ?it/s]

train evaluation:   0%|          | 0/3829 [00:00<?, ?it/s]

validation evaluation:   0%|          | 0/3829 [00:00<?, ?it/s]

train metrics:      {'mse': 0.1314029836680348, 'mae_stars': 1.3792965188758683}
validation metrics: {'mse': 0.13657375377232925, 'mae_stars': 1.4070892150142418}


Model(
  (movies_weights): Embedding(8670, 100)
  (users_weights): Embedding(22222, 100)
)